In [121]:
import re
from typing import Tuple
import numpy as np
from SolvingAdvent.HelferWichtel import HelferWichtel as Wichtel
from IPython.display import clear_output
import time

TagFile = '10'


Helfer = Wichtel()
PuzzleInput = ''.join(Helfer.getPuzzleInput(TagFile))
PuzzleInput = [line for line in PuzzleInput.split('\n')]
print(PuzzleInput)


['addx 1', 'noop', 'addx 4', 'noop', 'noop', 'noop', 'addx 6', 'addx -1', 'addx 5', 'noop', 'noop', 'noop', 'addx 5', 'addx -14', 'noop', 'addx 19', 'noop', 'addx 1', 'addx 4', 'addx 1', 'noop', 'noop', 'addx 2', 'addx 5', 'addx -27', 'addx 20', 'addx -30', 'addx 2', 'addx 5', 'addx 2', 'addx 4', 'addx -3', 'addx 2', 'addx 5', 'addx 2', 'addx -9', 'addx 1', 'addx 11', 'noop', 'addx -20', 'addx 7', 'addx 23', 'addx 2', 'addx 3', 'addx -2', 'addx -34', 'addx -2', 'noop', 'addx 3', 'noop', 'addx 3', 'addx 2', 'noop', 'addx 3', 'addx 2', 'addx 5', 'addx 2', 'addx -9', 'addx -7', 'addx 21', 'noop', 'addx 8', 'noop', 'addx -1', 'addx 3', 'addx -2', 'addx 5', 'addx -37', 'noop', 'addx 35', 'addx -31', 'addx 1', 'addx 4', 'addx -1', 'addx 2', 'noop', 'addx 3', 'addx 1', 'addx 5', 'addx -2', 'addx 7', 'addx -2', 'addx -2', 'addx 10', 'noop', 'addx 4', 'noop', 'noop', 'addx -19', 'addx 20', 'addx -38', 'noop', 'noop', 'addx 7', 'addx 2', 'addx 3', 'noop', 'addx 4', 'addx -3', 'addx 2', 'addx 2',

In [160]:
class commandParser:
    commands = {'addx', 'noop'}
    
    def tokenize(self,input):
        command = None
        argument = None
        command = re.search('[a-zA-Z]+',input).group()
        try:
            argument = re.search('-*\d+',input).group()
        except:
            argument = None
        return command, argument
        


class CRT:
    HEIGHT = 6
    WIDTH = 40
    
    def __init__(self):
        self.cycle = 0
        self.row = 0
        self.column = 0
        self.spritePosition = 0
        self.clearScreen()
        
    def setSpritePosition(self,position):
        self.spritePosition = position
        
    def moveCurrentPixel(self):
        if self.column == CRT.WIDTH-1:
            self.column = 0
            if self.row == CRT.HEIGHT-1:
                self.row = 0
                self.printScreen()
                self.clearScreen()
            else:
                self.row += 1
        else:
            self.column += 1
    
    def clearScreen(self):
        self.screen = [''.join(['.' for column in range(0,CRT.WIDTH)])  for row in range(0,CRT.HEIGHT)]
    
    def printScreen(self):
        clear_output(wait=True)
        print('\n'.join(self.screen))
        time.sleep(1)
        
    def addCycle(self):
        self.drawPixel()
        self.moveCurrentPixel()
        self.cycle += 1
    
    def drawPixel(self):
        row = self.row
        column = self.column
        if abs(self.column - self.spritePosition) <=1:
            self.screen[row] = self.screen[row][0:column] + '#' + self.screen[row][column+1:]
        else:
            self.screen[row] = self.screen[row][0:column] + '.' + self.screen[row][column+1:]
            
            
    
class Rechner:
    def __init__(self):
        self.cycles = 0
        self.register = 1
        self.interrupt = -1
        self.signalStrengthList = []
        self.CRT = CRT()
        
    def getRegister(self):
        return self.register
    
    def setRegister(self, newValue):
        self.register = newValue
        
    def getCycles(self):
        return self.cycles
    
    def getSignalStrengthList(self):
        return self.signalStrengthList
    
    def setNextCyclicInterupt(self):
        self.interrupt += self.periodicInterrupt
    
    def setUpInterrupt(self, firstInterrupt, periodicInterrupt):
        self.interrupt = firstInterrupt
        self.periodicInterrupt = periodicInterrupt
        
    def addCycle(self):
        self.cycles += 1
        self.CRT.addCycle()
        if bool(self.cycles == self.interrupt):
            self.interruptRoutine()
            self.setNextCyclicInterupt()

    def commandInput(self, command, argument):
        if command == 'noop':
            self.executeNoOp()
        if command == 'addx':
            self.executeAddition(argument)

    def executeAddition(self, value):
        self.addCycle()
        self.addCycle()
        self.setRegister(self.getRegister() + int(value))
        self.CRT.setSpritePosition(self.getRegister())
        
    def executeNoOp(self):
        self.addCycle()
        
    def calcSignalStrength(self):
        return self.getRegister()*self.cycles
        
    def interruptRoutine(self):
        signalStrength = self.calcSignalStrength()
        self.signalStrengthList.append(signalStrength)
        pass

In [161]:
# Class testing
Parser = commandParser()
newRechner = Rechner()
newRechner.setUpInterrupt(20,40)

for line in PuzzleInput:
    Token = Parser.tokenize(line)
    newRechner.commandInput(Token[0],Token[1])
print('Antwort A von Tag 10: %s' %sum(newRechner.getSignalStrengthList()))


####.###...##..###..#....####.####.#..#.
...#.#..#.#..#.#..#.#....#.......#.#..#.
..#..#..#.#..#.#..#.#....###....#..#..#.
.#...###..####.###..#....#.....#...#..#.
#....#.#..#..#.#.#..#....#....#....#..#.
####.#..#.#..#.#..#.####.#....####..##..
Antwort A von Tag 10: 14220


In [49]:
a = 'Testen, ob ich hiermit'
a.strip()
b = re.finditer('[^\s,]+',a)
for c in b:
    start = int(c.start())
    end = int(c.end())
    print(a[start:end])

Testen
ob
ich
hiermit


In [60]:
re.search('-?\d+','adb4 -44')

<re.Match object; span=(3, 4), match='4'>

In [86]:
-20%20

0

In [124]:
a = []
b = [''.join(['.' for column in range(0,40)])  for row in range(0,6)]
print('\n'.join(b))
time.sleep(1)
print(b[1][1])

........................................
........................................
........................................
........................................
........................................
........................................
.


In [152]:
a = 'abc'
a = a[0:1] + 'f' + a[2:]
print(a)

afc


In [159]:
a[3:]

''